In [37]:
import logging
import os
import time
from logging import Logger
from typing import List

import numpy as np

from symb_regression.config import GeneticParams
from symb_regression.core import GeneticProgram
from symb_regression.core.tree import Node
from symb_regression.utils.plotting import (
    plot_evolution_metrics,
    plot_prediction_analysis,
    plot_expression_tree,
)
from symb_regression.utils.data_handler import load_data
from symb_regression.utils.metrics import Metrics
import matplotlib.pyplot as plt

In [38]:
def print_section_header(title: str, logger: Logger | None = None):
    if logger is not None:
        logger.info("=" * 50)
        logger.info(f" {title} ".center(50, "="))
        logger.info("=" * 50)
        return
    else:
        print("\n" + "=" * 50)
        print(f" {title} ".center(50, "="))
        print("=" * 50)


def print_section_footer(logger: Logger | None = None):
    if logger is not None:
        logger.info("=" * 50 + "\n")
    else:
        print("=" * 50 + "\n")

In [39]:
def run_symbolic_regression(
    x: np.ndarray,
    y: np.ndarray,
    params: GeneticParams | None = None,
    debug: bool = False,
) -> tuple[Node, List[Metrics]]:
    logger: Logger = logging.getLogger("symb_regression")

    if params is None:
        params = GeneticParams(
            tournament_size=7,
            mutation_prob=0.4,
            crossover_prob=0.8,
            elitism_count=8,
            population_size=1000,
            generations=800,
            max_depth=5,
            min_depth=2,
        )

    if debug:
        logger.debug("Genetic Programming Parameters:")
        for key, value in params.__dict__.items():
            logger.debug(f"{key}: {value}")

    gp = GeneticProgram(params)

    logger.info("Starting evolution...")
    start_time = time.perf_counter()

    try:
        best_solution, history = gp.evolve(x, y)

        end_time = time.perf_counter()
        execution_time = end_time - start_time

        # Use print for better visibility of results
        print_section_header("SYMBOLIC REGRESSION RESULTS")
        print(f"Best Expression Found: {best_solution}")
        print(f"Final Fitness: {gp.calculate_fitness(best_solution, x, y):g}")
        print(f"Execution Time: {execution_time:.2f} seconds")
        print(f"Generations: {len(history)}")
        print_section_footer()

        # Plot the evolution progress

        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        plot_evolution_metrics(history, ax=axs[0])

        mse, r2 = plot_prediction_analysis(best_solution, x, y, ax=axs[1])

        plt.tight_layout()
        plt.show()

        print("Performance Metrics:")
        print(f"Mean Squared Error: {mse:.6f}")
        print(f"R² Score: {r2:.6f} ({r2*100:.1f}% of variance explained)")

        plot_expression_tree(best_solution)

        return best_solution, history

    except Exception as e:
        logger.error(f"Evolution failed: {e}")
        raise

In [40]:
# Set random seed for reproducibility
np.random.seed(42)

# Load and process data
PROBLEM_DIR = os.getcwd()
DATA_DIR = os.path.join(PROBLEM_DIR, "data")

x, y = load_data(DATA_DIR, "problem_4")

# Run symbolic regression
run_symbolic_regression(x, y)

Output()


Data Statistics:
X shape: (5000, 2), Y shape: (5000,)
Number of variables: 2

Variable x0:
  Range: [-4.999, 4.997]
  Mean: 0.009
  Std: 2.878
  Correlation with y: -0.049

Variable x1:
  Range: [-5.000, 5.000]
  Mean: -0.026
  Std: 2.865
  Correlation with y: -0.003

Target y:
  Range: [-4.158, 10.702]
  Mean: 2.000
  Std: 4.650


No improvement for 10 generations, injecting diversity...

No improvement for 10 generations, injecting diversity...

No improvement for 10 generations, injecting diversity...

...

No improvement for 10 generations, injecting diversity...

No improvement for 10 generations, injecting diversity...

No improvement for 10 generations, injecting diversity...

No improvement for 10 generations, injecting diversity...

KeyboardInterrupt: 